## Script to analyze RTS InTrain experiments and create outputs

### Imports 

In [ ]:
import warnings
from pathlib import Path

import pandas as pd
from utils import *
from utils_plotting import *

warnings.simplefilter(action='ignore', category=FutureWarning)

### Settings / Constants 

In [ ]:
DATA_DIR = Path('../02_UserResults')
DATA_BYK = Path('Bykovsky_digitized.shp')
DATA_PEEL = Path('Peel_digitized.shp')

RASTER_PEEL = Path('../01_Setup/Peel/data/PlanetScope_4763844_0870513_2021-08-04_2416_BGRN_SR.tif')
RASTER_BYK = Path('../01_Setup/Bykovsky/data/PlanetScope_4713120_5272315_2021-07-21_2463_BGRN_SR.tif')

FIG_DIR = Path('figures')
os.makedirs(FIG_DIR, exist_ok=True)

TOTAL_AREA = 2500**2

flist_byk = list(DATA_DIR.glob(f'*/{DATA_BYK}'))
flist_peel = list(DATA_DIR.glob(f'*/{DATA_PEEL}'))

### Load Data 

### Select Site here

In [ ]:
#site = 'Peel'
site = 'Bykovsky'

In [ ]:
if site == 'Peel':
    flist = flist_peel
    raster = RASTER_PEEL
elif site == 'Bykovsky':
    flist = flist_byk
    raster = RASTER_BYK

# define file for number of overlaps
overlap_file = Path(f'{site}_digitized_overlap.gpkg')

# create dir for figures
(FIG_DIR / site).mkdir(exist_ok=True)

#### Read files

In [ ]:
# read shape/file files to gpd DataFrames
dfs = read_files2(flist=flist, area=site)

# created df with dissolved data
joined = pd.concat([df.dissolve() for df in dfs]).sort_values('creator_id').reset_index(drop=True)

creators = joined['creator_id']

#### Plot correlation matrix
* set figut background color to white
* align names on xlabel 

In [ ]:
HALF = True
iou_array, df_metrics = create_iou_matrix_joined(joined, half=True, drop_same=False)

In [ ]:
# save to use and merge in illustrator
figure_path = FIG_DIR / site / f'{site}_IoUMatrix.png'
if figure_path is not None:
    os.makedirs(figure_path.parent, exist_ok=True)
plot_iou_matrix(site, iou_array, creators, save_path=figure_path, title=False, half=HALF, colorbar=False, fontcolor_change=0.6, transparent=False)

#### Table output

In [ ]:
# calculate metrics
HALF = True
iou_array_half, df_metrics_half = create_iou_matrix_joined(joined, half=HALF, drop_same=True)

In [ ]:
# calculate single objects
df_metrics_half_single = df_metrics_half.drop_duplicates(subset=['creator_1'])

In [ ]:
sorted_metrics_styled = style_results_table(df_metrics_half)
correlation_table_path = FIG_DIR / site / f'{site}_correlation_summary.html'
sorted_metrics_styled.to_html(correlation_table_path)

### Basic stats

#### Individual objects

In [ ]:
# mode of number of features
pd.value_counts(df_metrics_half_single['df1_number'])

#### Group stats

In [ ]:
mean_values = df_metrics_half.mean().round(2)
std_deviation = df_metrics_half.std().round(2)

#### Output raster
* single result

In [ ]:
# set properties
kwargs_norm=dict(p_low=1, p_high=97)
kwargs_fig_image=dict(figsize=(8,8), dpi=300)

####  Save Base image 

In [ ]:
transparent = True
save_path = FIG_DIR / site / f'{site}_321.png'
show_image(raster, save_path=save_path, title=None, kwargs_fig_image=kwargs_fig_image, kwargs_norm=kwargs_norm, bands=[3,2,1], transparent=transparent)

save_path = FIG_DIR / site / f'{site}_432.png'
show_image(raster, save_path=save_path, title=None, kwargs_fig_image=kwargs_fig_image, kwargs_norm=kwargs_norm, bands=[4,3,2], transparent=transparent)

### Plot label outlines over Raster

In [ ]:
target_directory = FIG_DIR / site / 'CIR'
show_raster = True
os.makedirs(target_directory, exist_ok=True)
plot_results_individual(joined, raster, site, target_directory, kwargs_fig_image=kwargs_fig_image, 
                        kwargs_norm=kwargs_norm, bands=[4,3,2], 
                        edgecolor='k', facecolor=(0,0,0,0.2), linewidth=1,
                        show_raster=show_raster)

In [ ]:
target_directory = FIG_DIR / site / 'RGB'
show_raster = True
os.makedirs(target_directory, exist_ok=True)
plot_results_individual(joined, raster, site, target_directory, kwargs_fig_image=kwargs_fig_image, 
                        kwargs_norm=kwargs_norm, bands=[3,2,1], 
                        edgecolor='k', facecolor=(0,0,0,0.2), linewidth=1,
                        show_raster=show_raster)

### Compare against each other visually 

#### Plot differences
* plot each combination

In [ ]:
# bg alpha = 0.2
fig_dir = FIG_DIR / site / 'diff' / 'RGBalpha02'
plot_diff(raster, plot_data, creators, site, fig_dir, kwargs_fig_image, kwargs_norm, 
          raster_alpha=0.2, bands=[3,2,1], show_title=False, save_fig=True, facecolor_intersection='yellow')

# bg alpha = 0.2
fig_dir = FIG_DIR / site / 'diff' / 'CIRalpha02'
plot_diff(raster, plot_data, creators, site, fig_dir, kwargs_fig_image, kwargs_norm, 
          raster_alpha=0.2, bands=[4,3,2], show_title=False, save_fig=True, facecolor_intersection='yellow')

#### Plot Consensus

In [ ]:
target_image_dir = FIG_DIR / site
save_path = target_image_dir / f'Consensus_map_{site}.png'

In [ ]:
kwargs_plot=dict(edgecolor=(0.2,0.2,0.2,1), lw=1)
show_consensus(image_path=raster, gdf=joined, 
               kwargs_norm=kwargs_norm, kwargs_plot=kwargs_plot,
               save_path=save_path, transparent=False)